# Roll Your Own Analysis: Debiasing Raw Date Exposure Data

This notebook will demonstrate how to download raw data from the ENPA REST API, debias date exposure data, and display a basic visualization by performing the following actions:

1. <a href="#Downloading-Raw-ENPA-Data">Downloading raw ENPA data</a>, 
2. <a href="#Debiasing-Raw-Data">Debiasing raw date exposure data</a>, and
3. <a href="#Visualizing-the-Debiased-Data">Visualizing the debiased data</a>.

Be sure to have your full API key ready before proceeding.

## Downloading Raw ENPA Data

This section will describe how to download raw ENPA data to perform further analysis using your full API key. Begin by following the steps below.

1) Navigate to the *API Docs* tab within the ENPA portal, and select *Authorize*.

![Step4](https://raw.githubusercontent.com/c19hcc/enpa-pha-jupyternotebooks/main/images/Step4.png)

2) Input your *Full Key* into the dialog box and select *Authorize*. Note, the full API key must be included as an `X-Api-Key` header in all requests.

![Step5W](https://raw.githubusercontent.com/c19hcc/enpa-pha-jupyternotebooks/main/images/Step5W.png)

3) Import Python's Request library and construct the necessary header to include the API key.

In [ ]:
import requests

header = {"X-Api-Key": # " insert API key here between the quotation marks "}

4) Specify parameters and request the desired data.
* `datasets`: Choose from one of several datasets to download :`notification`, `notificationInteractions`, `riskParameters`, `codeVerified`, `keysUploaded`, `beaconCount`, `dateExposure`, `dateExposure14d`, `dateExposureV2`, `keysUploadedVaccineStatus` , `keysUploadedVaccineStatus14d`, `keysUploadedVaccineStatusV2`, `keysUploadedWithReportType`, `keysUploadedWithReportType14d`, `periodicExposureNotification`, `periodicExposureNotification14d`, `secondaryAttack`, `secondaryAttack14d`, or `userRisk`.
* `raw`: Specify `True` to return the raw, biased data or `False` to return the debiased data.
* `start_date`: Specify a start date to download data (must be in YYYY-MM-DD format); if omitted, the default is 90 days before either the end date (if specified) or the current date.
* `end_date`: Specify an end date to download data (must be in YYYY-MM-DD format); if omitted, the default is the current date. 
* `country`: Input an ISO 3166-1 code for the country (e.g., `US`).
* `state`: Input an ISO-3166-2 code for the subdivision, state, or province (e.g., `US-VA`).

In [ ]:
parameters = {
    "datasets": "dateExposure14d",
    "raw": True,
    "start_date": "2022-03-31",
    "end_date": "2022-04-06",
    "country": "US",
    "state": "US-VA"
}

In [ ]:
DERequest = requests.get("https://api.enpa-pha.io/api/public/v2/enpa-data",
                      headers = header, params = parameters)
print("Reason: ", DERequest.reason, "\nStatus Code: ", DERequest.status_code)

A *Status Code* of 200 indicates that the data was successfully retrieved!

## Debiasing Raw Data

This section will describe how to prepare and debias the raw data downloaded from ENPA's REST API. Begin by following the steps below.

1) Convert the downloaded data from string to dictionary form using the `json` library, so it can be easy manipulated.

In [ ]:
import json
convertedDE = json.loads(DERequest.text)

Here is what the data look like. Note that `DateExposure` uses a cumulative distribution for the values. For each exposure notification classification, there are 12 buckets. For example, for classification 1 notifications, buckets 0-11 are used:
* Bucket 0 - delay is 0 days
* Bucket 1 - delay is 0-1 days
* Bucket 2 - delay is 0-2 days
* ...
* Bucket 10 - delay is 0-10 days
* Bucket 11 - any delay

Similarly, for a classification 2 exposure, buckets 12-23 are used. Classification 3 uses buckets 24-35, and classification 4 uses buckets 36-47.

In [ ]:
convertedDE

2) Crawling through the dictionary structure, we need to find the `totalCount`, the total number of clients (`total_individual_clients`); `sum`, the sum value (sum of an individual element of the `sum` array(s)); and `epsilon`, the epsilon from the raw value. These values are needed to debias the data. Here is what the debias function looks like.

In [ ]:
import math

def getMostLikelyPopulationCount(totalCount, sumPart, epsilon):
    '''
    Debiases raw sum values (`sumPart`).
    
    :param totalCount: The total number of clients (`total_individual_clients`)
    :param sumPart: The sum value (sum of an individual element of the `sum` array(s))
    :param epsilon: Epsilon from the raw value
    :return: 
        - `mostLikelyPopulation` (float): The debiased (expected) value
        - `standardDeviation` (float): Standard deviation
    '''
    p = 1 / (1 + math.exp(epsilon))
    sqrtPTimesOneMinusP = math.exp(epsilon / 2) / (1 + math.exp(epsilon))
    mostLikelyPopulation = (sumPart - totalCount * p) / (1 - 2 * p)
    standardDeviation = math.sqrt(totalCount) * sqrtPTimesOneMinusP
    return mostLikelyPopulation, standardDeviation

3) Let's debias the notification data by leveraging the debias function. We will create a dataframe aggregated by date with the debiased Apple and Google Notification data. Let's work with the first 7 bins (i.e., up to a 0-7 day delay) for type 1 notifications. Since this metric represents a percentage, we will need to include the last bin (bin 11) to serve as the divisor.

In [ ]:
from datetime import datetime
import pandas as pd

# Initialize Lists for Data Storage
datesApple, totalCountApple, epsilonApple, dateExposureApple = [], [], [], []
datesGoogle, totalCountGoogle, epsilonGoogle, dateExposureGoogle = [], [], [], []
SPG1, SPG2, SPG3, SPG4, SPG5, SPG6, SPG7, SPG8, SPGDivisor = [], [], [], [], [], [], [], [], []
SPA1, SPA2, SPA3, SPA4, SPA5, SPA6, SPA7, SPA8, SPADivisor = [], [], [], [], [], [], [], [], []

for i in range(len(convertedDE['rawData'])):
    # Data Collection to Debiasing Apple/iOS Data
    if(convertedDE['rawData'][i]['aggregation_id'] == "com.apple.EN.DateExposureV2D14"):
        datesApple.append(datetime.strptime(convertedDE['rawData'][i]['aggregation_start_time'][0:10], "%Y-%m-%d"))
        totalCountApple.append(convertedDE['rawData'][i]['total_individual_clients']) # totalCount parameter for debiasing
        SPA1.append(convertedDE['rawData'][i]['sum'][0]) # sumPart 0 day delay
        SPA2.append(convertedDE['rawData'][i]['sum'][1]) # sumPart 0-1 day delay
        SPA3.append(convertedDE['rawData'][i]['sum'][2]) # sumPart 0-2 day delay
        SPA4.append(convertedDE['rawData'][i]['sum'][3]) # sumPart 0-3 day delay
        SPA5.append(convertedDE['rawData'][i]['sum'][4]) # sumPart 0-4 day delay
        SPA6.append(convertedDE['rawData'][i]['sum'][5]) # sumPart 0-5 day delay
        SPA7.append(convertedDE['rawData'][i]['sum'][6]) # sumPart 0-6 day delay
        SPA8.append(convertedDE['rawData'][i]['sum'][7]) # sumPart 0-7 day delay
        SPADivisor.append(convertedDE['rawData'][i]['sum'][11]) # total cumulative sumPart delay
        epsilonApple.append(convertedDE['rawData'][i]['epsilon']) # epsilon parameter for debiasing
        
    # Data Collection to Debiasing Google/Android Data
    elif(convertedDE['rawData'][i]['aggregation_id'] == "DateExposure14d"):
        datesGoogle.append(datetime.strptime(convertedDE['rawData'][i]['aggregation_start_time'][0:10], "%Y-%m-%d"))
        totalCountGoogle.append(convertedDE['rawData'][i]['total_individual_clients']) # totalCount parameter for debiasing
        SPG1.append(convertedDE['rawData'][i]['sum'][0]) # sumPart 0 day delay
        SPG2.append(convertedDE['rawData'][i]['sum'][1]) # sumPart 0-1 day delay
        SPG3.append(convertedDE['rawData'][i]['sum'][2]) # sumPart 0-2 day delay
        SPG4.append(convertedDE['rawData'][i]['sum'][3]) # sumPart 0-3 day delay
        SPG5.append(convertedDE['rawData'][i]['sum'][4]) # sumPart 0-4 day delay
        SPG6.append(convertedDE['rawData'][i]['sum'][5]) # sumPart 0-5 day delay
        SPG7.append(convertedDE['rawData'][i]['sum'][6]) # sumPart 0-6 day delay
        SPG8.append(convertedDE['rawData'][i]['sum'][7]) # sumPart 0-7 day delay
        SPGDivisor.append(convertedDE['rawData'][i]['sum'][11]) # total cumulative sumPart delay
        epsilonGoogle.append(convertedDE['rawData'][i]['epsilon']) # epsilon parameter for debiasing

# Place Apple data in a dataframe.
RawAppleData = pd.DataFrame(list(zip(datesApple, totalCountApple, SPA1, SPA2, SPA3, SPA4, SPA5, SPA6, 
                                     SPA7, SPA8, SPADivisor, epsilonApple)), 
                            columns = ['Date', 'Total', 'SumPartApple0', 'SumPartApple1', 'SumPartApple2',
                                       'SumPartApple3','SumPartApple4', 'SumPartApple5','SumPartApple6', 
                                       'SumPartApple7', 'SumPartDivisor', 'Epsilon'])        

# Aggregate the sumPart bins for each day for the Apple. 
# Epsilon is segregated since it should not be summed.
epsPart = ['Epsilon']
sumParts = RawAppleData.columns.difference(epsPart).tolist()

# Perform Groupby Operations on Apple Data
g = RawAppleData.groupby(['Date'])
sumPartsGrouped = g[sumParts].sum()
epsPartGrouped = g[epsPart].mean() # Epsilon will be constant throughout the day.
RawAppleGrouping = pd.concat([sumPartsGrouped, epsPartGrouped], axis = 1)
  
# Place Google data in a dataframe.
RawGoogleData = pd.DataFrame(list(zip(datesGoogle, totalCountGoogle, SPG1, SPG2, SPG3, SPG4, SPG5, SPG6, 
                                      SPG7, SPG8, SPGDivisor, epsilonGoogle)), 
                             columns = ['Date', 'Total', 'SumPartGoogle0', 'SumPartGoogle1', 'SumPartGoogle2',
                                        'SumPartGoogle3','SumPartGoogle4', 'SumPartGoogle5','SumPartGoogle6', 
                                        'SumPartGoogle7', 'SumPartDivisor', 'Epsilon'])     

# Aggregate the sumPart bins for each day for the Google data. 
# Epsilon is segregated since it should not be summed.
epsPart = ['Epsilon']
sumParts = RawGoogleData.columns.difference(epsPart).tolist()

# Perform Groupby Operations on Apple Data
g = RawGoogleData.groupby(['Date'])
sumPartsGrouped = g[sumParts].sum()
epsPartGrouped = g[epsPart].mean() # Epsilon will be constant throughout the day.
RawGoogleGrouping = pd.concat([sumPartsGrouped, epsPartGrouped], axis = 1)

# Debias the Apple Notification Data
for i in range(RawAppleGrouping.shape[0]):
    temp = []
    for c in range(0, 9):
        sumPart = RawAppleGrouping.iloc[i,c]
        (mostLikelyDE, standardDeviation) = getMostLikelyPopulationCount(RawAppleGrouping.Total[i], 
                                                                         sumPart, RawAppleGrouping.Epsilon[i])
        temp.append(mostLikelyDE)
    dateExposureApple.append(temp)

# Debias the Google Notification Data
for i in range(RawGoogleGrouping.shape[0]):
    temp = []
    for c in range(0, 9):
        sumPart = RawGoogleGrouping.iloc[i,c]
        (mostLikelyDE, standardDeviation) = getMostLikelyPopulationCount(RawGoogleGrouping.Total[i], 
                                                                         sumPart, RawGoogleGrouping.Epsilon[i])
        temp.append(mostLikelyDE)
    dateExposureGoogle.append(temp)

# Clean Up Data
RawAppleGrouping['DebiasedAppleDE'] = dateExposureApple
RawGoogleGrouping['DebiasedGoogleDE'] = dateExposureGoogle
DebiasedData = pd.DataFrame(list(zip(RawAppleGrouping.index, RawAppleGrouping.DebiasedAppleDE, 
                                     RawGoogleGrouping.DebiasedGoogleDE)),
                           columns = ['Date', 'Debiased Apple Date Exposure', 'Debiased Google Date Exposure'])

Here is what the debiased data look like:

In [ ]:
pd.set_option('display.max_colwidth', 200)
DebiasedData

4) All that is left is combining the datasets appropriately. Begin by combining the Apple and Google data.

In [ ]:
TotalDE = []
for r in range(DebiasedData.shape[0]):
    totalStaging = []
    for i in range(0, len(DebiasedData.iloc[r, 1])):
        totalStaging.append(DebiasedData.iloc[r, 1][i] + DebiasedData.iloc[r, 2][i])
    TotalDE.append(totalStaging)

Here are what the finalized data look like.

In [ ]:
MergedData = DebiasedData.copy()
MergedData['Combined Date Exposure'] = TotalDE
MergedData

Finally divide each bin by the last bin, the divisor. 

In [ ]:
# Partition the bins in the dataframe
DateExposure = pd.DataFrame(MergedData['Combined Date Exposure'].to_list(), columns=['Raw Bin 0', 'Raw Bin 1',
                                                                                     'Raw Bin 2', 'Raw Bin 3', 
                                                                                     'Raw Bin 4', 'Raw Bin 5',
                                                                                     'Raw Bin 6', 'Raw Bin 7',
                                                                                     'Raw Bin 11'])
# Calculate the cumulative percentages
DateExposure['Bin 0'] = DateExposure['Raw Bin 0'] / DateExposure['Raw Bin 11'] * 100
DateExposure['Bin 1'] = DateExposure['Raw Bin 1'] / DateExposure['Raw Bin 11'] * 100
DateExposure['Bin 2'] = DateExposure['Raw Bin 2'] / DateExposure['Raw Bin 11'] * 100
DateExposure['Bin 3'] = DateExposure['Raw Bin 3'] / DateExposure['Raw Bin 11'] * 100
DateExposure['Bin 4'] = DateExposure['Raw Bin 4'] / DateExposure['Raw Bin 11'] * 100
DateExposure['Bin 5'] = DateExposure['Raw Bin 5'] / DateExposure['Raw Bin 11'] * 100
DateExposure['Bin 6'] = DateExposure['Raw Bin 6'] / DateExposure['Raw Bin 11'] * 100
DateExposure['Bin 7'] = DateExposure['Raw Bin 7'] / DateExposure['Raw Bin 11'] * 100
DateExposure = DateExposure.mask(DateExposure < 0, 0)
DateExposure['Date'] = MergedData['Date'].dt.strftime('%m/%d/%Y')
DateExposure

## Visualizing the Debiased Data

This section will describe how to prepare and visualize the raw data downloaded from ENPA's REST API. Begin by following the steps below.

1) Run the code below to construct the bar chart. The dataframe, `DateExposure`, is already organized, so it is easy to plot the data.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

# Create the base axis to add the bars
fig, ax = plt.subplots(1, 1, figsize=(20, 8))

# Font size of the x tick label
plt.rc('xtick', labelsize=11) 

# Extract the date for the x labels
label = DateExposure['Date']
x = np.arange(len(label))

# Specifies the bar width
width = 0.22

# Create bars and offset them, so they are not displayed on top of one another
rect0 = ax.bar(x - width, DateExposure['Bin 2'], width = width, label = "0-2 Day Delay", edgecolor = "black")
rect1 = ax.bar(x, DateExposure['Bin 4'], width = width, label = "0-4 Day Delay", edgecolor = "black")
rect2 = ax.bar(x + width, DateExposure['Bin 6'], width = width, label = "0-6 Day Delay", edgecolor = "black")

# Add labels
ax.set_ylabel("Percent of Type 1 Notifications", fontsize = 20)
ax.set_xlabel("Date", fontsize = 20)
ax.set_title("Date Exposure: Percent of Type 1 Notifications", fontsize = 25)

# Use the date as the x labels
ax.set_xticks(x)
ax.set_xticklabels(label)
ax.tick_params(axis = "x", which = "both", labelrotation = 65)

# Set the legend font size
ax.legend(fontsize = 14)

# Add gridlines and an axis line
ax.grid(which='major', linestyle=':', linewidth='0.2', color='black')
plt.axhline(0, color='black', ls='solid');

2) Next, save the plot to your current directory. The filename will include the date range of the plot (e.g., `DateExposurePlot_2022-01-01_to_2022-01-31.pdf`).

In [ ]:
startDate = datetime.strftime(MergedData['Date'][0], "%Y-%m-%d")
endDate = datetime.strftime(MergedData['Date'][len(MergedData['Date']) - 1], "%Y-%m-%d")
fig.tight_layout()
ax.figure.savefig(f'DateExposurePlot_{startDate}_to_{endDate}.pdf', format='pdf', dpi=400)